In [ ]:
from os.path import exists
from urllib.request import urlretrieve
import pandas as pd

In [ ]:
url = "https://dati.comune.milano.it/dataset/845682d9-d0c9-41a5-8bbe-f28dc668a1cd/resource/9e226d9f-ad8b-47e8-a217-88db4673797b/download/v2_accessiorari_disaggregazionecategoria_euroareacundefined_2015.json"
file = 'data/hourly2015.json'
if not exists(file):
    urlretrieve(url, file)
df = pd.read_json(file)
df['datetime'] = pd.to_datetime(df['data'] + ' ' + df['timeIndex'])
df.set_index('datetime', inplace=True)
df

In [ ]:
df.EURO4.plot()

In [ ]:
def load_daily_json(url, file):
    if not exists(file):
        urlretrieve(url, file)
    df = pd.read_json(file)
    df.rename(columns={'data_giorno':'timeIndex', 
                       'numero_transiti_giornalieri':'totale'}, inplace=True)
    try: 
        df['datetime'] = pd.to_datetime(df['timeIndex'])
    except:
        df['datetime'] = pd.to_datetime(df['data_giorno'])
    df.set_index('datetime', inplace=True)
    return df


files = {
    'data/daily2012.json': "https://dati.comune.milano.it/dataset/55a1dbf1-0442-48df-bffe-4f2615b1c473/resource/0ca542c9-0d83-44b0-948f-8fd877443078/download/v2_accessigiornalieri_areacundefined_2012.json",
    'data/daily2013.json': "https://dati.comune.milano.it/dataset/b597f3b0-1616-4117-97c4-64e1d619edac/resource/b906728b-ecab-4ff9-b30b-968a3e3a3aa0/download/v2_accessigiornalieri_areacundefined_2013.json",
    'data/daily2013.json': "https://dati.comune.milano.it/dataset/b597f3b0-1616-4117-97c4-64e1d619edac/resource/b906728b-ecab-4ff9-b30b-968a3e3a3aa0/download/v2_accessigiornalieri_areacundefined_2013.json",
    'data/daily2014.json': "https://dati.comune.milano.it/dataset/0083deed-5af1-46d1-a603-0d10f1500fc6/resource/2af44a47-d12e-42c7-aca5-b0a08431ba00/download/v2_accessigiornalieri_areacundefined_2014.json",
    'data/daily2015.json': "https://dati.comune.milano.it/dataset/134141e6-5e39-4900-ad3f-4d7c5bf411e3/resource/74cd4c73-6748-46cb-8972-0476ffaa550b/download/v2_accessigiornalieri_areacundefined_2015.json",
    'data/daily2016-18.json': "https://dati.comune.milano.it/dataset/792cfd91-6cea-40d3-bcc2-feda97c110e4/resource/fcf2a4c2-1867-47ed-94db-8e7898d65236/download/ds1085_ingressi_areac_precedenti.json",
    'data/daily2019-23.json': "https://dati.comune.milano.it/dataset/8937eb87-2356-40ba-bd82-e0fabe38b598/resource/c16a1f83-57b8-473e-b2ba-e626baa4db7a/download/ingressi_areac_2023-04-19.json"
}

df = pd.DataFrame()
for f, url in files.items():
    df = df.append(load_daily_json(url, f))

df

In [ ]:
years = [i for i in range(2012, 2024)]
years

In [ ]:
import numpy as np
import hvplot.pandas 
from holoviews import opts, dim, Layout

In [ ]:
Layout([pd.DataFrame(df[df.index.year==i]).hvplot.heatmap(
        title=f'AREA C Daily access {i}', 
        x='datetime.day', y='datetime.month', 
        C='totale', reduce_function=np.sum) for i in years]
).cols(1)

In [ ]:
Layout([pd.DataFrame(df[df.index.year==i]).hvplot.heatmap(
        title=f'AREA C Daily access {i}', 
        x='datetime.week', y='datetime.weekday', 
        C='totale', reduce_function=np.sum) for i in years]
).cols(1)

In [ ]:
s = df.totale.resample("1d").sum()

In [ ]:
s.plot()

In [ ]:
from adtk.data import validate_series
from adtk.visualization import plot
s = validate_series(s)
s

In [ ]:
from adtk.detector import ThresholdAD
threshold_ad = ThresholdAD(high=180000, low=25000)
anomalies = threshold_ad.detect(s)
plot(s, anomaly=anomalies, ts_linewidth=1, ts_markersize=3, anomaly_markersize=5, anomaly_color='red', anomaly_tag="marker");

In [ ]:
from adtk.detector import QuantileAD
quantile_ad = QuantileAD(high=0.95, low=0.05)
anomalies = quantile_ad.fit_detect(s)
plot(s, anomaly=anomalies, ts_linewidth=1, ts_markersize=3, anomaly_markersize=5, anomaly_color='red', anomaly_tag="marker");

In [ ]:
from adtk.transformer import ClassicSeasonalDecomposition
s_transformed = ClassicSeasonalDecomposition(freq=7, trend=True).fit_transform(s).rename("Seasonal decomposition residual")
plot(pd.concat([s, s_transformed], axis=1), ts_linewidth=1, ts_markersize=4);

In [ ]:
from adtk.detector import SeasonalAD
seasonal_ad = SeasonalAD(c=3.0, side="both")
anomalies = seasonal_ad.fit_detect(s)
plot(s, anomaly=anomalies, anomaly_color="red", anomaly_tag="marker")

In [ ]:
from adtk.transformer import RollingAggregate
s_transformed = RollingAggregate(agg='sum', window=7).transform(s)
plot(s_transformed.rename("Rolling count of valid values"), ts_linewidth=1, ts_markersize=4);

In [ ]:
from adtk.detector import SeasonalAD
seasonal_ad = SeasonalAD(c=3.0, side="both")
anomalies = seasonal_ad.fit_detect(s_transformed)
plot(s_transformed, anomaly=anomalies, anomaly_color="red", anomaly_tag="marker")